커피를 마시는 사람과 마시지 않는 사람의 키를 비교해겠습니다. Data set에는 21살 미만/이상 여부, 커
피를 마시는지 마시지 않는지 여부, 그리고 키에 대한 데이터가 있습니다. 부트스트랩 연습을 위해 모수를 모른다고 가정하고 200개의 샘플에서 부트스트래핑을 할 것입니다. 커피가 키에 영향을 미치는지 분석하기 위해 아래 4개의 분석을 하겠습니다. 처음에 200개의 샘플을 뽑고, 그것을 활용하여 모두 부트스트래핑을 합니다.
1. 커피를 마시지 않는 사람과 커피를 마시는 사람의 평균 키 차이
2. 21살 이상과 21살 미만인 사람들의 평균 키 차이
3. 21살 미만인 사람들 중 커피를 마시지 않는 사람과 커피를 마시는 사람의 평균 키 차이
4. 21살 이상인 사람들 중 커피를 마시지 않는 사람과 커피를 마시는 사람의 평균 키 차이

### 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
np.random.seed(104)

import matplotlib.pyplot as plt

%matplotlib inline

df = pd.read_csv('coffee_dataset.csv')

In [2]:
df.head()

,user_id,age,drinks_coffee,height
0,4509,<21,False,64.538179
1,1864,>=21,True,65.824249
2,2060,<21,False,71.319854
3,7875,>=21,True,68.569404
4,6254,<21,True,64.020226


모집단에서 200개의 표본을 랜덤샘플링합니다.

In [3]:
df_sample = df.sample(200)
df_sample.head()

,user_id,age,drinks_coffee,height
486,4951,<21,False,68.539632
2297,5383,>=21,True,70.594166
911,3904,<21,False,61.114035
146,4351,>=21,True,68.952444
2255,3998,<21,False,65.493818


### 부트스트랩(Bootstrap)
부트스트랩을 10,000번 반복해서 커피를 마시지 않는 사람과 마시는 사람의 키 차이의 99% 신뢰구간을 구합니다.

1. 커피를 마시지 않는 사람과 커피를 마시는 사람의 평균 키 차이

In [4]:
iterationNum = 10000
diffHeightList = []
for _ in range(iterationNum):
    bootSample = df_sample.sample(200, replace=True) # 복원 추출
    nonCoffeeHeightMean = bootSample[bootSample['drinks_coffee'] == False].height.mean() # 커피를 마시지 않는 사람 평균 키
    coffeeHeightMean = bootSample[bootSample['drinks_coffee'] == True].height.mean() # 커피를 마시는 사람 평균 키
    diff = nonCoffeeHeightMean - coffeeHeightMean
    diffHeightList.append(diff)

#  신뢰수준 99%인 평균 키 차이에 대한 신뢰구간
np.percentile(diffHeightList, 0.5), np.percentile(diffHeightList, 99.5)

(-3.1319975931075725, -0.7811253958264867)

신뢰수준 99%로 커피를 마시지 않는 사람이 커피를 마시는 사람에 비해 더 작습니다. 나이에 따라 분석이 달라질 수 있으니, 21살 미만, 이상의 키 차이를 구해보겠습니다.

2. 21살 이상과 21살 미만인 사람들의 평균 키 차이

In [5]:
diffHeightListByAge = []
for _ in range(iterationNum):
    bootSample = df_sample.sample(200, replace=True) # 복원 추출
    over21HeightMean = bootSample[bootSample['age'] == '>=21'].height.mean() # 21살 이상 평균 키
    under21HeightMean = bootSample[bootSample['age'] == '<21'].height.mean() # 21살 미만 평균 키
    diff = over21HeightMean - under21HeightMean
    diffHeightListByAge.append(diff)

np.percentile(diffHeightListByAge, 0.5), np.percentile(diffHeightListByAge, 99.5)

(3.094910696764073, 4.97970620735041)

신뢰수준 99%로 21살 이상이 21살 미만에 비해 키가 큽니다. 21살 미만인 사람들 중 커피를 마시지 않는 사람과 마시는 사람의 평균 키를 비교해보겠습니다.

3. 21살 미만인 사람들 중 커피를 마시지 않는 사람과 커피를 마시는 사람의 평균 키 차이

In [6]:
diffHeightListUnder21 = []
for _ in range(iterationNum):
    bootSample = df_sample.sample(200, replace=True) # 복원 추출
    nonCoffeeHeightMeanUnder21 = bootSample.query("age == '<21' and drinks_coffee == False").height.mean() # 21살 미만이며 커피를 마시지 않는 사람 평균 키
    coffeeHeightMeanUnder21 = bootSample.query("age == '<21' and drinks_coffee == True").height.mean() # 21살 미만이며 커피를 마시는 사람 평균 키
    diff = nonCoffeeHeightMeanUnder21 - coffeeHeightMeanUnder21
    diffHeightListUnder21.append(diff)

np.percentile(diffHeightListUnder21, 0.5), np.percentile(diffHeightListUnder21, 99.5)

(0.265445115542466, 2.473977523788757)

4. 21살 이상인 사람들 중 커피를 마시지 않는 사람과 커피를 마시는 사람의 평균 키 차이

In [18]:
diffHeightListOver21 = []
for _ in range(iterationNum):
    bootSample = df_sample.sample(200, replace=True) # 복원 추출
    nonCoffeeHeightMeanOver21 = bootSample.query("age != '<21' and drinks_coffee == False").height.mean() # 21살 이상이며 커피를 마시지 않는 사람 평균 키
    coffeeHeightMeanOver21 = bootSample.query("age != '<21' and drinks_coffee == True").height.mean() # 21살 이상이며 커피를 마시는 사람 평균 키
    
    diff = nonCoffeeHeightMeanOver21 - coffeeHeightMeanOver21
    diffHeightListOver21.append(diff)

np.percentile(diffHeightListOver21, 0.5), np.percentile(diffHeightListOver21, 99.5)

(0.38466991879827656, 3.290024158015461)

### 모수와 부트스트랩 신뢰구간의 비교
원본 데이터를 알고 있기 때문에 실제 평균의 차이를 구해서 신뢰구간 안에 있는지 보겠습니다.

In [21]:
# 1. 커피를 마시지 않는 사람과 커피를 마시는 사람의 평균 키 차이
df[df['drinks_coffee'] == False].height.mean() - df[df['drinks_coffee'] == True].height.mean()

-1.9568024933369657

In [22]:
# 2. 21살 이상과 21살 미만인 사람들의 평균 키 차이
df[df['age'] == '>=21'].height.mean() - df[df['age'] == '<21'].height.mean()

3.88229124992111

In [23]:
# 3. 21살 미만인 사람들 중 커피를 마시지 않는 사람과 커피를 마시는 사람의 평균 키 차이
df.query("age == '<21' and drinks_coffee == False").height.mean() - df.query("age == '<21' and drinks_coffee == True").height.mean()

1.6993900935511732

In [26]:
# 4. 21살 이상인 사람들 중 커피를 마시지 않는 사람과 커피를 마시는 사람의 평균 키 차이
df.query("age != '<21' and drinks_coffee == False").height.mean() - df.query("age != '<21' and drinks_coffee == True").height.mean()

1.9509354889786579